# Import packages

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns 

from sklearn.metrics import silhouette_score, silhouette_samples
import sklearn.metrics
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, log_loss
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

import itertools

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import scipy

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display
pd.options.display.max_columns = None

# Load data

In [2]:
df=pd.read_csv('/System/Volumes/Data/Users/Michael/Library/CloudStorage/iCloud Drive/MMA/MMA 831/allstate-purchase-prediction-challenge/train.csv')

# Check the data

In [3]:
# pandas_profiling.ProfileReport(df, check_correlation=False)

In [13]:
df.head(10)

,customer_ID,shopping_pt,record_type,day,time,state,location,group_size,homeowner,car_age,car_value,risk_factor,age_oldest,age_youngest,married_couple,C_previous,duration_previous,A,B,C,D,E,F,G,cost
0,10000000,1,0,0,08:35,IN,10001,2,0,2,g,3.0,46,42,1,1.0,2.0,1,0,2,2,1,2,2,633
1,10000000,2,0,0,08:38,IN,10001,2,0,2,g,3.0,46,42,1,1.0,2.0,1,0,2,2,1,2,1,630
2,10000000,3,0,0,08:38,IN,10001,2,0,2,g,3.0,46,42,1,1.0,2.0,1,0,2,2,1,2,1,630
3,10000000,4,0,0,08:39,IN,10001,2,0,2,g,3.0,46,42,1,1.0,2.0,1,0,2,2,1,2,1,630
4,10000000,5,0,0,11:55,IN,10001,2,0,2,g,3.0,46,42,1,1.0,2.0,1,0,2,2,1,2,1,630
5,10000000,6,0,0,11:57,IN,10001,2,0,2,g,3.0,46,42,1,1.0,2.0,1,0,2,2,1,2,1,638
6,10000000,7,0,0,11:58,IN,10001,2,0,2,g,3.0,46,42,1,1.0,2.0,1,0,2,2,1,2,1,638
7,10000000,8,0,0,12:03,IN,10001,2,0,2,g,3.0,46,42,1,1.0,2.0,1,0,2,2,1,2,1,638
8,10000000,9,1,0,12:07,IN,10001,2,0,2,g,3.0,46,42,1,1.0,2.0,1,0,2,2,1,2,1,634
9,10000005,1,0,3,08:56,NY,10006,1,0,10,e,4.0,28,28,0,3.0,13.0,1,1,3,3,1,0,2,755


# Slice the data and keep only the purchase record

In [5]:
df_purchased=df[df.record_type==1]
df_purchased.head()
df_purchased.info()

,customer_ID,shopping_pt,record_type,day,time,state,location,group_size,homeowner,car_age,car_value,risk_factor,age_oldest,age_youngest,married_couple,C_previous,duration_previous,A,B,C,D,E,F,G,cost
8,10000000,9,1,0,12:07,IN,10001,2,0,2,g,3.0,46,42,1,1.0,2.0,1,0,2,2,1,2,1,634
14,10000005,6,1,3,09:09,NY,10006,1,0,10,e,4.0,28,28,0,3.0,13.0,0,0,3,2,0,0,2,731
22,10000007,8,1,4,14:26,PA,10008,1,0,11,c,NaN,43,43,0,2.0,4.0,0,0,1,2,0,0,1,602
26,10000013,4,1,4,09:31,WV,10014,2,1,3,d,3.0,62,60,1,3.0,3.0,1,1,3,2,1,1,3,626
32,10000014,6,1,1,17:50,MO,10015,1,0,5,d,3.0,32,28,1,1.0,2.0,1,1,1,1,0,2,2,614


<class 'pandas.core.frame.DataFrame'>
Int64Index: 97009 entries, 8 to 665248
Data columns (total 25 columns):
customer_ID          97009 non-null int64
shopping_pt          97009 non-null int64
record_type          97009 non-null int64
day                  97009 non-null int64
time                 97009 non-null object
state                97009 non-null object
location             97009 non-null int64
group_size           97009 non-null int64
homeowner            97009 non-null int64
car_age              97009 non-null int64
car_value            97009 non-null object
risk_factor          62663 non-null float64
age_oldest           97009 non-null int64
age_youngest         97009 non-null int64
married_couple       97009 non-null int64
C_previous           96173 non-null float64
duration_previous    96173 non-null float64
A                    97009 non-null int64
B                    97009 non-null int64
C                    97009 non-null int64
D                    97009 non-null int64

# Create the purchase record table

In [14]:
list_product=["A","B","C","D","E","F","G"]
df_prducct=pd.DataFrame()
for i in list_product:
    df_prducct=pd.concat([df_prducct,pd.get_dummies(df_purchased[i],prefix=i)],1)
df_prducct.head()

,A_0,A_1,A_2,B_0,B_1,C_1,C_2,C_3,C_4,D_1,D_2,D_3,E_0,E_1,F_0,F_1,F_2,F_3,G_1,G_2,G_3,G_4
8,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0
14,1,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0
22,1,0,0,1,0,1,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0
26,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0
32,0,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0


# Calculate the support

In [15]:
%time frequent_itemsets = apriori(df_prducct, min_support=0.001, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()
frequent_itemsets.tail()

CPU times: user 12.4 s, sys: 10.2 s, total: 22.6 s
Wall time: 19.8 s


,support,itemsets,length
0,0.220371,(A_0),1
1,0.614190,(A_1),1
2,0.165438,(A_2),1
3,0.523910,(B_0),1
4,0.476090,(B_1),1


,support,itemsets,length
7990,0.003031,"(A_2, C_3, G_3, E_1, D_3, F_2, B_1)",7
7991,0.001165,"(A_2, F_3, C_3, E_1, G_2, D_3, B_1)",7
7992,0.001361,"(A_2, F_3, C_3, G_3, E_1, D_3, B_1)",7
7993,0.001278,"(A_2, C_4, E_1, G_2, D_3, F_0, B_1)",7
7994,0.001072,"(A_2, C_4, G_3, E_1, D_3, F_2, B_1)",7


In [26]:
frequent_itemsets_list=frequent_itemsets[ (frequent_itemsets['length'] == 2) & (frequent_itemsets['support'] >= 0.02) ]
frequent_itemsets_list.head()

,support,itemsets,length
22,0.154841,"(B_0, A_0)",2
23,0.065530,"(A_0, B_1)",2
24,0.105691,"(A_0, C_1)",2
25,0.051892,"(A_0, C_2)",2
26,0.054211,"(C_3, A_0)",2


# Calculate the support and confidence

In [17]:
%time rules = association_rules(frequent_itemsets, min_threshold=0.1)

CPU times: user 1.33 s, sys: 68.3 ms, total: 1.4 s
Wall time: 916 ms


In [18]:
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(B_0),(A_0),0.523910,0.220371,0.154841,0.295549,1.341143,0.039387,1.106719,1
1,(A_0),(B_0),0.220371,0.523910,0.154841,0.702638,1.341143,0.039387,1.601046,1
2,(A_0),(B_1),0.220371,0.476090,0.065530,0.297362,0.624592,-0.039387,0.745633,1
3,(B_1),(A_0),0.476090,0.220371,0.065530,0.137642,0.624592,-0.039387,0.904066,1
4,(A_0),(C_1),0.220371,0.298869,0.105691,0.479605,1.604733,0.039829,1.347306,1


# Slice the data

***review what we have learnd in MMA869***
🤣🤣🤣🤣

![title](/System/Volumes/Data/Users/Michael/Library/CloudStorage/iCloud Drive/MMA/MMA 831/WX20190728-164045@2x.png)

In [74]:
sliced=rules[ (rules['antecedent_len'] >= 1) &
       (rules['confidence'] > 0.85) &
       (rules['lift'] > 1.2) & 
        (rules['support'] > 0.15)]

In [75]:
sliced_sort_by_confidence=sliced.sort_values('lift',ascending=False)

In [76]:
sliced_sort_by_confidence
sliced_sort_by_confidence.shape

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
675,(A_0),"(E_0, F_0)",0.220371,0.232752,0.190477,0.864347,3.713601,0.139185,5.655944,1
672,"(A_0, E_0)",(F_0),0.214372,0.322094,0.190477,0.888536,2.758626,0.121429,6.081854,2
18,(A_0),(F_0),0.220371,0.322094,0.192539,0.873702,2.712570,0.121559,5.367510,1
673,"(A_0, F_0)",(E_0),0.192539,0.537393,0.190477,0.989292,1.840909,0.087008,43.202827,2
383,"(B_0, A_0)",(E_0),0.154841,0.537393,0.152934,0.987684,1.837916,0.069724,37.561152,2
16,(A_0),(E_0),0.220371,0.537393,0.214372,0.972776,1.810174,0.095946,16.992445,1
1281,"(F_2, E_1)",(A_1),0.205259,0.614190,0.179427,0.874146,1.423250,0.053358,3.065539,2
2445,"(C_3, B_1)",(D_3),0.194704,0.646404,0.178839,0.918520,1.420969,0.052982,4.339651,2
8704,"(C_3, A_1, E_1)",(D_3),0.173077,0.646404,0.158563,0.916141,1.417288,0.046685,4.216533,3
3278,"(C_3, E_1)",(D_3),0.210145,0.646404,0.192271,0.914942,1.415433,0.056432,4.157099,2


(15, 10)